# COCO

Возьмем датасет COCO и разделим его на две группы: 
1) книга есть на фото
2) книга на фото нет

In [2]:
import os

while os.getcwd().split("/")[-1] != "book_counter":
    os.chdir(os.path.abspath(os.path.join(os.getcwd(), "..")))

In [ ]:
import random
import shutil

import joblib
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
from PIL import Image
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from torch.utils.data import DataLoader, Dataset
from torchvision import models, transforms
from tqdm import tqdm

In [4]:
def organize_books_dataset(source_dir, target_dir):
    book_classes = ["book_jacket", "bookcase", "bookshop", "comic_book"]

    books_dir = os.path.join(target_dir, "books")
    not_books_dir = os.path.join(target_dir, "not_books")

    os.makedirs(books_dir, exist_ok=True)
    os.makedirs(not_books_dir, exist_ok=True)

    book_files = []
    other_files = []

    for class_name in os.listdir(source_dir):
        class_path = os.path.join(source_dir, class_name)

        if not os.path.isdir(class_path):
            continue

        image_files = []
        for file in os.listdir(class_path):
            if file.lower().endswith((".jpg", ".jpeg", ".png", ".bmp", ".tiff")):
                image_files.append(os.path.join(class_path, file))

        if class_name in book_classes:
            book_files.extend(image_files)
        else:
            other_files.extend(image_files)

    for file_path in book_files:
        filename = os.path.basename(file_path)
        class_name = os.path.basename(os.path.dirname(file_path))
        new_filename = f"{class_name}_{filename}"
        target_path = os.path.join(books_dir, new_filename)
        shutil.copy2(file_path, target_path)

    if len(other_files) >= len(book_files):
        selected_other_files = random.sample(other_files, len(book_files))
    else:
        selected_other_files = other_files

    for file_path in selected_other_files:
        filename = os.path.basename(file_path)
        class_name = os.path.basename(os.path.dirname(file_path))
        new_filename = f"{class_name}_{filename}"
        target_path = os.path.join(not_books_dir, new_filename)
        shutil.copy2(file_path, target_path)

In [5]:
source_dataset = "dataset"
target_dataset = "organized_dataset"

organize_books_dataset(source_dataset, target_dataset)